In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# Load the FAISS index and metadata
index = faiss.read_index('vector_store/faiss_index.index')
metadata_df = pd.read_csv('vector_store/metadata.csv')

# Load the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Initialize the language model for generation
generator = pipeline('text-generation', model='gpt-3.5-turbo')

def retrieve_relevant_chunks(question, k=5):
    question_embedding = model.encode([question])
    distances, indices = index.search(np.array(question_embedding, dtype=np.float32), k)
    
    retrieved_chunks = []
    for idx in indices[0]:
        if idx != -1:  # Check for valid index
            retrieved_chunks.append(metadata_df.iloc[idx]['original_id'])
    
    return retrieved_chunks

def generate_answer(question, context):
    prompt_template = (
        "You are a financial analyst assistant for CrediTrust. "
        "Your task is to answer questions about customer complaints. "
        "Use the following retrieved complaint excerpts to formulate your answer. "
        "If the context doesn't contain the answer, state that you don't have enough information. "
        "Context: {context} "
        "Question: {question} "
        "Answer:"
    )
    
    context_str = "\n".join(context)
    prompt = prompt_template.format(context=context_str, question=question)
    
    response = generator(prompt, max_length=150)
    
    return response[0]['generated_text'], context_str

def chatbot(question):
    context = retrieve_relevant_chunks(question)
    answer, sources = generate_answer(question, context)
    return answer, sources

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## CrediTrust Complaint Analysis Assistant")
    
    question_input = gr.Textbox(label="Type your question here:")
    submit_button = gr.Button("Ask")
    
    answer_output = gr.Markdown(label="Generated Answer:")
    sources_output = gr.Markdown(label="Source Chunks:")
    
    submit_button.click(chatbot, inputs=question_input, outputs=[answer_output, sources_output])
    
    clear_button = gr.Button("Clear")
    clear_button.click(lambda: ("", ""), outputs=[answer_output, sources_output])

# Launch the Gradio app
demo.launch()